In [1]:
import duckdb
import pandas as pd

con = duckdb.connect()
con.execute("SET temp_directory='data/processed/tmp';")
con.execute("PRAGMA threads=4;")
con.execute("PRAGMA memory_limit='1GB';")

con.execute("""
CREATE OR REPLACE VIEW events AS
SELECT * FROM read_parquet([
  'data/processed/events_2019_oct.parquet',
  'data/processed/events_2019_nov.parquet'
]);
""")

con.execute("""
CREATE OR REPLACE VIEW events_enriched AS
SELECT
  *,
  DATE(event_time) AS event_date,
  STRFTIME(event_time, '%Y-%m') AS event_month,
  STRFTIME(event_time, '%Y-%W') AS event_week
FROM events;
""")


In [2]:
con.execute("""
SELECT event_month, COUNT(*) AS events
FROM events_enriched
GROUP BY event_month
ORDER BY event_month
""").df()


,event_month,events
0,2019-10,42448764
1,2019-11,67501979


In [3]:
funnel_by_month = con.execute("""
WITH per_user_month AS (
  SELECT
    event_month,
    user_id,
    MAX(CASE WHEN event_type='view' THEN 1 ELSE 0 END) AS did_view,
    MAX(CASE WHEN event_type='cart' THEN 1 ELSE 0 END) AS did_cart,
    MAX(CASE WHEN event_type='purchase' THEN 1 ELSE 0 END) AS did_purchase
  FROM events_enriched
  GROUP BY event_month, user_id
)
SELECT
  event_month,
  COUNT(*) AS total_users,
  SUM(did_view) AS view_users,
  SUM(did_cart) AS cart_users,
  SUM(did_purchase) AS purchase_users,
  ROUND(SUM(did_cart)*1.0 / NULLIF(SUM(did_view),0), 4) AS view_to_cart_rate,
  ROUND(SUM(did_purchase)*1.0 / NULLIF(SUM(did_cart),0), 4) AS cart_to_purchase_rate,
  ROUND(SUM(did_purchase)*1.0 / NULLIF(SUM(did_view),0), 4) AS view_to_purchase_rate
FROM per_user_month
GROUP BY event_month
ORDER BY event_month;
""").df()

funnel_by_month


,event_month,total_users,view_users,cart_users,purchase_users,view_to_cart_rate,cart_to_purchase_rate,view_to_purchase_rate
0,2019-10,3022290,3022130.0,337117.0,347118.0,0.1115,1.0297,0.1149
1,2019-11,3696117,3695598.0,826323.0,441638.0,0.2236,0.5345,0.1195


In [4]:
price_funnel = con.execute("""
WITH base AS (
  SELECT
    event_month,
    user_id,
    CASE 
      WHEN price < 20 THEN 'low'
      WHEN price BETWEEN 20 AND 100 THEN 'mid'
      ELSE 'high'
    END AS price_segment,
    MAX(CASE WHEN event_type='view' THEN 1 ELSE 0 END) AS did_view,
    MAX(CASE WHEN event_type='cart' THEN 1 ELSE 0 END) AS did_cart,
    MAX(CASE WHEN event_type='purchase' THEN 1 ELSE 0 END) AS did_purchase
  FROM events_enriched
  WHERE price IS NOT NULL
  GROUP BY event_month, user_id, price_segment
)
SELECT
  event_month,
  price_segment,
  SUM(did_view) AS viewers,
  SUM(did_cart) AS carters,
  SUM(did_purchase) AS buyers,
  ROUND(SUM(did_cart)*1.0 / NULLIF(SUM(did_view),0), 4) AS view_to_cart_rate,
  ROUND(SUM(did_purchase)*1.0 / NULLIF(SUM(did_view),0), 4) AS view_to_purchase_rate
FROM base
GROUP BY event_month, price_segment
ORDER BY event_month, price_segment;
""").df()

price_funnel


,event_month,price_segment,viewers,carters,buyers,view_to_cart_rate,view_to_purchase_rate
0,2019-10,high,2510760.0,273284.0,252549.0,0.1088,0.1006
1,2019-10,low,527919.0,10841.0,19782.0,0.0205,0.0375
2,2019-10,mid,1489304.0,76936.0,115603.0,0.0517,0.0776
3,2019-11,high,3042200.0,617107.0,323579.0,0.2028,0.1064
4,2019-11,low,728216.0,62066.0,23180.0,0.0852,0.0318
5,2019-11,mid,1918554.0,294823.0,151482.0,0.1537,0.0790


In [5]:
category_funnel = con.execute("""
WITH base AS (
  SELECT
    event_month,
    user_id,
    category_code,
    MAX(CASE WHEN event_type='view' THEN 1 ELSE 0 END) AS did_view,
    MAX(CASE WHEN event_type='cart' THEN 1 ELSE 0 END) AS did_cart,
    MAX(CASE WHEN event_type='purchase' THEN 1 ELSE 0 END) AS did_purchase
  FROM events_enriched
  WHERE category_code IS NOT NULL
  GROUP BY event_month, user_id, category_code
),
top_categories AS (
  SELECT category_code
  FROM base
  GROUP BY category_code
  ORDER BY SUM(did_view) DESC
  LIMIT 10
)
SELECT
  b.event_month,
  b.category_code,
  SUM(did_view) AS viewers,
  SUM(did_cart) AS carters,
  SUM(did_purchase) AS buyers,
  ROUND(SUM(did_cart)*1.0 / NULLIF(SUM(did_view),0), 4) AS view_to_cart_rate,
  ROUND(SUM(did_purchase)*1.0 / NULLIF(SUM(did_view),0), 4) AS view_to_purchase_rate
FROM base b
JOIN top_categories t ON b.category_code = t.category_code
GROUP BY b.event_month, b.category_code
ORDER BY b.event_month, viewers DESC;
""").df()

category_funnel


,event_month,category_code,viewers,carters,buyers,view_to_cart_rate,view_to_purchase_rate
0,2019-10,electronics.smartphone,1300070.0,198347.0,160437.0,0.1526,0.1234
1,2019-10,electronics.audio.headphone,213948.0,22482.0,18826.0,0.1051,0.0880
2,2019-10,electronics.clocks,211922.0,9481.0,10746.0,0.0447,0.0507
3,2019-10,electronics.video.tv,170044.0,16672.0,13476.0,0.0980,0.0793
4,2019-10,appliances.kitchen.washer,132028.0,9948.0,10480.0,0.0753,0.0794
5,2019-10,appliances.kitchen.refrigerators,131602.0,6477.0,7656.0,0.0492,0.0582
6,2019-10,computers.notebook,129207.0,7474.0,9185.0,0.0578,0.0711
7,2019-10,apparel.shoes,126760.0,0.0,2757.0,0.0000,0.0217
8,2019-10,appliances.environment.vacuum,119925.0,8527.0,9191.0,0.0711,0.0766
9,2019-10,apparel.shoes.keds,91620.0,0.0,1951.0,0.0000,0.0213
